In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)

import matplotlib
matplotlib.rcParams['figure.figsize'] = (15,7)

1. Import Data

In [ ]:
data = pd.read_csv('../input/heart-disease-uci/heart.csv')
data.head()

In [ ]:
print(data.shape, 'This dataset has', data.shape[0], 'rows &', data.shape[1], 'columns')

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sum()

2. Data Visualization

In [ ]:
sns.countplot(data['target'], palette='Set1')
plt.title('Patients who has and has not disease')
plt.xlabel('Target')
plt.ylabel('Number of patients')
plt.show()

*We can see that our dataset is balanced distributed: 140 patients has not disease & 163 has disease*

In [ ]:
sns.distplot(data['age'], bins = 20)
plt.title('Dataset Members Age Distribution')
plt.xlabel('Age')
plt.ylabel('Distribution')
plt.show()

In [ ]:
sns.boxplot(data['target'], data['age'], palette='Set1')
plt.title('Patients who has and has not disease depends on Age')
plt.xlabel('Target')
plt.ylabel('Age')
plt.show()

*We can see that patients who has disease varies from 45 years old to almost 60 years old and for people who has not disease are on age from 52 years old to 63 years old. Age is not a good predictor for this disease*

In [ ]:
sns.countplot(data['target'], hue = data['sex'], palette='Set1')
plt.title('Patients with or without disease depends on Sex')
plt.xlabel('Target')
plt.ylabel('Nr of patients')
plt.show()

In [ ]:
columns_feature = ['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

for i in columns_feature:
    sns.boxplot(data['target'], data[i], palette='Set1')
    plt.title('Target depends on ' + i)
    plt.xlabel('Target')
    plt.ylabel(i)
    plt.show()

3.Data Modeling

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve

In [ ]:
data.head()

In [ ]:
X = data.drop('target', axis = 1)
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [ ]:
logistic = LogisticRegression(random_state=0)
dt = DecisionTreeClassifier(random_state=1)
svc = SVC()
random = RandomForestClassifier(random_state=1)
xgb = XGBClassifier(random_state = 10)

Logistic Regression

In [ ]:
logistic.fit(X_train, list(y_train.values))
log_pred = logistic.predict(X_test)
print('Accuracy: ', logistic.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), log_pred))
print('Recall: ', recall_score(list(y_test.values), log_pred))
print('F1 Score: ', f1_score(list(y_test.values), log_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), log_pred))

Decision Tree

In [ ]:
dt.fit(X_train, list(y_train.values))
dt_pred = dt.predict(X_test)
print('Accuracy: ', dt.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), dt_pred))
print('Recall: ', recall_score(list(y_test.values), dt_pred))
print('F1 Score: ', f1_score(list(y_test.values), dt_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), dt_pred))

Support Vector 

In [ ]:
svc.fit(X_train, list(y_train.values))
svc_pred = svc.predict(X_test)
print('Accuracy: ', svc.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), svc_pred))
print('Recall: ', recall_score(list(y_test.values), svc_pred))
print('F1 Score: ', f1_score(list(y_test.values), svc_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), svc_pred))

Random Forest

In [ ]:
random.fit(X_train, list(y_train.values))
random_pred = random.predict(X_test)
print('Accuracy: ', random.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), random_pred))
print('Recall: ', recall_score(list(y_test.values), random_pred))
print('F1 Score: ', f1_score(list(y_test.values), random_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), random_pred))

XGBoost

In [ ]:
xgb.fit(X_train, list(y_train.values))
xgb_pred = xgb.predict(X_test)
print('Accuracy: ', xgb.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), xgb_pred))
print('Recall: ', recall_score(list(y_test.values), xgb_pred))
print('F1 Score: ', f1_score(list(y_test.values), xgb_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), xgb_pred))


For better comparison between the models let's draw the ROC curves

In [ ]:
y_score_log = logistic.decision_function(X_test)
y_score_dt = dt.predict_proba(X_test)[:,1]
y_score_svc = svc.decision_function(X_test)
y_score_random = random.predict_proba(X_test)[:,1]
y_score_xgb = xgb.predict_proba(X_test)[:,1]

fpr_log, tpr_log, _ = roc_curve(y_test.values, y_score_log)
fpr_dt, tpr_dt, _ = roc_curve(y_test.values, y_score_dt)
fpr_svc, tpr_svc, _ = roc_curve(y_test.values, y_score_svc)
fpr_random, tpr_random, _ = roc_curve(y_test.values, y_score_random)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test.values, y_score_xgb)

plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr_log, tpr_log, label = "Logistic Regression")
plt.plot(fpr_dt, tpr_dt, label = 'Decision Tree')
plt.plot(fpr_svc, tpr_svc, label = 'Support Vector')
plt.plot(fpr_random, tpr_random, label = 'Random Forest')
plt.plot(fpr_xgb, tpr_xgb, label = 'XGBoost')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
y_score_random = random.predict_proba(X_test)[:,1]
y_score_xgb = xgb.predict_proba(X_test)[:,1]

fpr_random, tpr_random, _ = roc_curve(y_test.values, y_score_random)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test.values, y_score_xgb)

plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr_random, tpr_random, label = 'Random Forest')
plt.plot(fpr_xgb, tpr_xgb, label = 'XGBoost')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

Let's rank the features in terms of importance

In [ ]:
importances = random.feature_importances_
std = np.std([tree.feature_importances_ for tree in random.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
# Print the feature ranking
print("Feature ranking:")

for f in range(X_test.shape[1]):
    print("%d. feature: %s (%f)" % (f + 1, data.iloc[:,indices[f]].name, importances[indices[f]]))

In [ ]:
labels = ['cp', 'ca', 'thalach', 'oldpeak', 'chol', 'age', 'thal', 'trestbps']

X2 = X[labels]

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size = 0.2, random_state = 10)

In [ ]:
random2 = RandomForestClassifier(random_state=1)
random2.fit(X_train2, list(y_train2.values))
random2_pred = random2.predict(X_test2)

print('Accuracy: ', random2.score(X_test2, list(y_test2.values)))
print('Precission: ', precision_score(list(y_test2.values), random2_pred))
print('Recall: ', recall_score(list(y_test2.values), random2_pred))
print('F1 Score: ', f1_score(list(y_test2.values), random2_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test2.values), random2_pred))

XGBoost Feature Importance

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [ ]:
feat_sel_model = SelectFromModel(Lasso(alpha = 0.005, random_state = 42))
feat_sel_model.fit(X, y)

In [ ]:
feat_sel_model.get_support()

In [ ]:
selected_feat = X.columns[feat_sel_model.get_support()]

In [ ]:
X3 = X[selected_feat]
X3.head()

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y, test_size = 0.2, random_state = 10)

In [ ]:
xgb3 = XGBClassifier(random_state = 10)
xgb3.fit(X_train3, list(y_train3.values))
xgb_pred3 = xgb3.predict(X_test3)
print('Accuracy: ', xgb3.score(X_test3, list(y_test3.values)))
print('Precission: ', precision_score(list(y_test3.values), xgb_pred3))
print('Recall: ', recall_score(list(y_test3.values), xgb_pred3))
print('F1 Score: ', f1_score(list(y_test3.values), xgb_pred3))
print('Confusion matrix: \n', confusion_matrix(list(y_test3.values), xgb_pred3))

*This is a very small dataset so when we remove columns we loose information that is why the accuracy is decreasing*

Random Forest Optimization Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 50)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3,
                              verbose = 2, random_state = 42, n_jobs = -1)
rf_random.fit(X_train2, y_train2)

In [ ]:
rf_random.best_estimator_

In [ ]:
random3 = RandomForestClassifier(max_depth=80, max_features='sqrt', min_samples_split=10,
                       n_estimators=991)
random3.fit(X_train2, list(y_train2.values))
random3_pred = random3.predict(X_test2)

print('Accuracy: ', random3.score(X_test2, list(y_test2.values)))
print('Precission: ', precision_score(y_test2.values, random3_pred))
print('Recall: ', recall_score(list(y_test2.values), random3_pred))
print('F1 Score: ', f1_score(list(y_test2.values), random3_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test2.values), random3_pred))

XGBoost Optimization Tuning

In [ ]:
n_estimators = [int (x) for x in np.linspace(100, 1500, num = 50)]
learning_rate = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
gamma = [0.0, 0.1, 0.2, 0.3, 0.4]
max_depth = [int(x) for x in np.linspace(1, 20, num = 10)]
min_child_weight = [int(x) for x in np.linspace(1, 5, num = 5)]
booster = ['gbtree', 'gblinear']
base_score = [0.25, 0.5, 0.75, 1]

hyper_parameter = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    'gamma': gamma,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'booster': booster,
    'base_score': base_score
}

In [ ]:
xgb_optimize = XGBClassifier()
xgb_random = RandomizedSearchCV(estimator=xgb_optimize, param_distributions=hyper_parameter, n_iter=100,
                                cv = 3, verbose=2, random_state=42, n_jobs=-1)
xgb_random.fit(X_train, y_train)

In [ ]:
xgb_random.best_estimator_

In [ ]:
xgb_optimize =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.3, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=414, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
xgb_optimize.fit(X_train, list(y_train.values))
xgb_optimize_pred = xgb_optimize.predict(X_test)
print('Accuracy: ', xgb_optimize.score(X_test, list(y_test.values)))
print('Precission: ', precision_score(list(y_test.values), xgb_optimize_pred))
print('Recall: ', recall_score(list(y_test.values), xgb_optimize_pred))
print('F1 Score: ', f1_score(list(y_test.values), xgb_optimize_pred))
print('Confusion matrix: \n', confusion_matrix(list(y_test.values), xgb_optimize_pred))

Random Forest after feature selection is the best model

In [ ]:
y_score_random = random.predict_proba(X_test)[:,1]
y_score_random2 = random2.predict_proba(X_test2)[:,1]
y_score_xgb = xgb.predict_proba(X_test)[:,1]

fpr_random, tpr_random, _ = roc_curve(y_test.values, y_score_random)
fpr_random2, tpr_random2, _ = roc_curve(y_test2.values, y_score_random2)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test.values, y_score_xgb)

plt.plot(fpr_random, tpr_random, label = 'Random Forest', color = 'orange')
plt.plot(fpr_random2, tpr_random2, label = 'Random Forest Optimize', color = 'blue')
plt.plot(fpr_xgb, tpr_xgb, label = 'XGBoost', color = 'green')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()